In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#필요한 라이브러리 import
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import torchvision.models as models
from sklearn.model_selection import train_test_split
import torch.optim as optim
from datetime import datetime
from sklearn.metrics import f1_score
import torch.nn.functional as F


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks
!mkdir USG_AI

/content/drive/MyDrive/Colab Notebooks
mkdir: cannot create directory ‘USG_AI’: File exists


In [ ]:
#working directory 옮기기
%cd /content/drive/MyDrive/Colab Notebooks/USG_AI
!ls
# !unzip -q -o "{MAIN_DIR}/task1.zip" #현재 디렉토리에 압축풀기

/content/drive/MyDrive/Colab Notebooks/USG_AI
cnn_metric.pth		  task1			     task1_J.ipynb
resnet101_checkpoint.pth  task1_20241110_053518.csv  task2
resnet101_metric.pth	  task1_20241110_054152.csv  task2_done_0932.csv
resnet18_checkpoint.pth   task1_20241110_135055.csv  task2_done_0936.csv
resnet18_metric.pth	  task1_20241110_140353.csv  task2_done_0939.csv
resnet50_checkpoint.pth   task1_20241111_081311.csv  task2_done_{current_time}.csv
resnet50_metric.pth	  task1_20241112_030253.csv


In [ ]:
MAIN_DIR = os.getcwd() #USG_AI까지의 경로
TASK1_DATA_DIR = MAIN_DIR + "/task1" #USG_AI/task1의 경로
TRAIN_DIR = TASK1_DATA_DIR + "/train" #task1/train
TEST_DIR = TASK1_DATA_DIR + "/test" #task1/test

In [ ]:
len(os.listdir(TRAIN_DIR)), len(os.listdir(TEST_DIR)) #압축해제된 파일 개수가 맞는지 확인

(3500, 1501)

In [ ]:
#개요에 명시된 test 파일 개수와 압축해제한 파일의 개수가 달라 확인

sample = pd.read_csv( TASK1_DATA_DIR+"/answer_sample.csv" )
file_name = sample["file_name"]
dir_list = os.listdir(TEST_DIR)
set(dir_list) - set(file_name) #.DS_Store 파일

{'.DS_Store'}

In [ ]:
train_labels = pd.read_csv(TASK1_DATA_DIR + "/train_labels.csv")
train_file = pd.DataFrame({"file_name" : train_labels["file_name"],
                           "file_path" : [TRAIN_DIR + "/" + i for i in os.listdir(TRAIN_DIR)],
                           "labels" : train_labels["bad"].astype("float")})
train_file # train data의 이름, 경로, 라벨을 묶어 하나의 dataFrame 만들기

,file_name,file_path,labels
0,train0000.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,0.0
1,train0001.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,0.0
2,train0002.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,0.0
3,train0003.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,1.0
4,train0004.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,1.0
...,...,...,...
3495,train3495.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,1.0
3496,train3496.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,0.0
3497,train3497.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,1.0
3498,train3498.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,1.0


In [ ]:
SEED = 42
np.random.seed(SEED)
status = ["train" if i <= 0.8 else "val" for i in np.random.uniform(low=0.0, high=1.0, size=(train_file.shape[0]))]
train_file["status"] = status
train_file["status"].value_counts(normalize = True)

,proportion
status,
train,0.800571
val,0.199429


In [ ]:
val_file = train_file[train_file["status"] == "val"]
val_file

,file_name,file_path,labels,status
1,train0001.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,0.0,val
7,train0007.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,0.0,val
11,train0011.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,1.0,val
12,train0012.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,0.0,val
33,train0033.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,0.0,val
...,...,...,...,...
3483,train3483.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,1.0,val
3492,train3492.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,1.0,val
3493,train3493.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,1.0,val
3496,train3496.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,0.0,val


In [ ]:
train_file = train_file[train_file["status"] == "train"]
train_file

,file_name,file_path,labels,status
0,train0000.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,0.0,train
2,train0002.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,0.0,train
3,train0003.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,1.0,train
4,train0004.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,1.0,train
5,train0005.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,1.0,train
...,...,...,...,...
3491,train3491.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,0.0,train
3494,train3494.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,1.0,train
3495,train3495.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,1.0,train
3498,train3498.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...,1.0,train


In [ ]:
test_file = pd.DataFrame({"file_name" : [i for i in os.listdir(TEST_DIR) if i != ".DS_Store" ],
                           "file_path" : [TEST_DIR + "/" + i for i in os.listdir(TEST_DIR) if i != ".DS_Store" ]})

test_file #test 파일이름, 경로 dataFrame


,file_name,file_path
0,test0505.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...
1,test0499.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...
2,test0506.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...
3,test0503.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...
4,test0501.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...
...,...,...
1495,test0487.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...
1496,test0497.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...
1497,test0507.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...
1498,test0496.jpeg,/content/drive/MyDrive/Colab Notebooks/USG_AI/...


In [ ]:
#데이터 셋 정의
class CustomDataset(Dataset):
    def __init__(self, dataframe, transform=None, include_labels=True):
        self.dataframe = dataframe
        self.transform = transform
        self.include_labels = include_labels

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        data_row = self.dataframe.iloc[idx]
        image_path = data_row["file_path"]
        image = Image.open(image_path).convert("RGB")

        if self.transform:
          image = self.transform(image)

        if self.include_labels:
            label = data_row["labels"]
            label = torch.tensor(label, dtype=torch.float32)
            return image, label
        else:
            return image

In [ ]:
#train_transform
#train_transform은 데이터 증강(data augmentation) 해야함
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),# 50% 확률로 좌우 반전
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(degrees=30),   # -15도에서 +15도 범위로 랜덤 회전
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # 밝기, 대비, 채도, 색조 조정
    transforms.ToTensor(),  # 텐서 변환
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


#test_transform 만들어야 함
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),                   # 텐서 변환
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


#Resize(224,224)
#ToTensor()만 하면 됨

# 데이터셋에 전처리 적용
BATCH_SIZE = 32
NUM_WORKERS = 4

train_dataset = CustomDataset(dataframe=train_file, transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

val_dataset = CustomDataset(dataframe = val_file, transform = test_transform)
val_loader = DataLoader(val_dataset, batch_size = BATCH_SIZE, num_workers=NUM_WORKERS)

test_dataset = CustomDataset(dataframe = test_file, transform = test_transform, include_labels=False)
test_loader = DataLoader(test_dataset, batch_size = BATCH_SIZE, num_workers=NUM_WORKERS)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
def show_rgb_images(loader, num_images=16):
    # DataLoader에서 한 배치 가져오기
    images, labels = next(iter(loader))

    # 4x4 그리드에 이미지 시각화
    grid_size = int(num_images ** 0.5)
    fig, axes = plt.subplots(grid_size, grid_size, figsize=(10, 10))

    for i, ax in enumerate(axes.flat):
        if i >= num_images:
            break
        image = images[i].permute(1, 2, 0).numpy()  # [C, H, W] -> [H, W, C]

        ax.imshow(image)
        ax.set_title(f"Label: {labels[i]}")
        ax.axis('off')  # 축 제거

    plt.tight_layout()
    plt.show()

# show_rgb_images(train_loader)


In [ ]:
# Checkpoint 저장 함수
def save_checkpoint(model, optimizer, epoch, checkpoint_path):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, checkpoint_path)


# Checkpoint 로드 함수
def load_checkpoint(model, optimizer, checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']

    return model, optimizer, epoch


def save_metric_checkpoint(train_losses, val_losses, val_accuracies , checkpoint_path):
    torch.save({
        'train_losses': train_losses,
        'val_losses': val_losses,
        'val_accuracies': val_accuracies,
    }, checkpoint_path)


def load_metric_checkpoint(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    train_losses = checkpoint['train_losses']
    val_losses = checkpoint['val_losses']
    val_accuracies = checkpoint['val_accuracies']

    return train_losses, val_losses, val_accuracies


def train_model(model, train_loader, val_loader, criterion, optimizer, device, start_epoch, num_epochs, checkpoint_path, metric_path):
    train_loss_list = []
    val_loss_list = []
    val_accuracy_list = []
    best_accuracy = 0.0

    for epoch in range(start_epoch, num_epochs):
        print(f"Start {epoch+1}")
        # Training phase
        model.train()
        running_loss = 0.0
        for i, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.view(-1, 1).to(device)

            optimizer.zero_grad()  # Gradient 초기화
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()  # 역전파
            optimizer.step()  # 가중치 업데이트

            running_loss += loss.item()

            # 매 100번째 배치마다 로그 출력
            if (i + 1) % 50 == 0:
                print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}")

        # 에포크 종료 후 평균 학습 손실 출력
        epoch_train_loss = running_loss / len(train_loader)

        print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_train_loss:.4f}")

        # Validation phase
        model.eval()  # 모델을 평가 모드로 설정
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():  # 검증 중에는 Gradient 계산 비활성화
            for images, labels in val_loader:
                images, labels = images.to(device), labels.view(-1, 1).to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                # 정확도 계산 (이진 분류 가정)
                predicted = (outputs > 0.5).float()  # Sigmoid 기준으로 예측
                correct += (predicted == labels).sum().item()
                total += labels.size(0)

        # 에폭 종료 후 검증 손실 및 정확도 계산
        epoch_val_loss = val_loss / len(val_loader)
        val_loss_list.append(epoch_val_loss)

        val_accuracy = correct / total
        val_accuracy_list.append(val_accuracy)

        # 검증 결과 출력
        print(f"Epoch [{epoch+1}/{num_epochs}], Validation Loss: {epoch_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

        #val_accyracy가 최고점수일 경우 모델 파라미터 저장
        if val_accuracy > best_accuracy:
            train_loss_list.append(epoch_train_loss)
            val_loss_list.append(epoch_val_loss)
            val_accuracy_list.append(val_accuracy)
            best_accuracy = val_accuracy

            save_checkpoint(model, optimizer, epoch, checkpoint_path)
            save_metric_checkpoint(train_loss_list,val_loss_list, val_accuracy_list, metric_path)
            print("Model saved\n")

    return train_loss_list, val_loss_list, val_accuracy_list





#test
def predict(model, test_loader):
    model.eval()
    image_names = []
    predictions = []

    with torch.no_grad():
        for images in test_loader:

            outputs = model(images)
            predicted = (outputs > 0.5).squeeze()

            predicted = predicted.int()

            batch_filenames = test_loader.dataset.dataframe['file_name'][:len(images)]
            batch_filenames = test_loader.dataset.dataframe.loc[
                test_loader.dataset.dataframe.index[:len(images)], 'file_name'
            ]
            image_names.extend(batch_filenames)
            predictions.extend(predicted.numpy().tolist())


    results_df = pd.DataFrame({
        'file_name': image_names,
        'bad': predictions
    })

    return results_df



In [ ]:
class CNNmodel(nn.Module):
  def __init__(self):
    super(CNNmodel, self).__init__()
    self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
    self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
    self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
    self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)

    self.fc1 = nn.Linear(128 * 14 * 14, 256)
    self.fc2 = nn.Linear(256, 1)

    self.bn1 = nn.BatchNorm2d(16)
    self.bn2 = nn.BatchNorm2d(32)
    self.bn3 = nn.BatchNorm2d(64)
    self.bn4 = nn.BatchNorm2d(128)

  def forward(self, x):
    x = F.relu(self.bn1(self.conv1(x)))
    x = F.max_pool2d(x, 2)
    x = F.relu(self.bn2(self.conv2(x)))
    x = F.max_pool2d(x, 2)
    x = F.relu(self.bn3(self.conv3(x)))
    x = F.max_pool2d(x, 2)
    x = F.relu(self.bn4(self.conv4(x)))
    x = F.max_pool2d(x, 2)


    x = x.view(x.size(0), -1)
    x = F.relu(self.fc1(x))
    x = torch.sigmoid(self.fc2(x))
    return x

In [ ]:
class CNNmodel2(nn.Module):
    def __init__(self):
        super(CNNmodel2, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)

        self.fc1 = nn.Linear(128 * 224 * 224, 256)
        self.fc2 = nn.Linear(256, 1)

        self.bn1 = nn.BatchNorm2d(16)
        self.bn2 = nn.BatchNorm2d(32)
        self.bn3 = nn.BatchNorm2d(64)
        self.bn4 = nn.BatchNorm2d(128)

        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # Max Pooling을 사용 않고 stride와 배치 정규화
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))


        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc2(x))
        return x

In [ ]:
model = CNNmodel2()


"""
# 모든 파라미터를 고정 (Gradient 계산 비활성화)
for param in model.parameters():
    param.requires_grad = False

# Layer3와 Layer4의 파라미터만 학습 가능하도록 설정
for param in model.layer3.parameters():
    param.requires_grad = True

for param in model.layer4.parameters():
    param.requires_grad = True

num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(num_features, 1),
    nn.Sigmoid()
)

"""
LR = 0.0001
NUM_EPOCH = 4
MODEL_NAME = "CNNmodel2"

criterion = nn.BCELoss()  # 이진교차
optimizer = optim.Adam(model.parameters(), lr=LR)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 학습 및 테스트
checkpoint_path = MAIN_DIR + f'/{MODEL_NAME}_checkpoint.pth'
metric_path = MAIN_DIR + f'/{MODEL_NAME}_metric.pth'

train_loss_list = []
val_loss_list = []
val_accuracy_list = []

# 성능 지표 로드 (이전에 저장한 지표가 있는 경우)
if os.path.exists(metric_path):
    train_loss_list, val_loss_list, val_accuracy_list = load_metric_checkpoint(metric_path)
    print(f"Loaded metric from  {metric_path}")


# Checkpoint 로드 (이전에 학습한 모델이 있는 경우)
"""
if os.path.exists(checkpoint_path):
    model, optimizer, START_EPOCH = load_checkpoint(model, optimizer, checkpoint_path)
    print(f"Loaded checkpoint from epoch {START_EPOCH}")"""

if os.path.exists(checkpoint_path):
    os.remove(checkpoint_path)  # 이전 체크포인트 파일 삭제
    START_EPOCH = 0
    print("Checkpoint not compatible with new model structure, starting from scratch.")

else:
    START_EPOCH = 0
    print("Do not exist checkpoint")



t_loss, v_loss, v_acc = train_model(model, train_loader, val_loader, criterion, optimizer, device, START_EPOCH, NUM_EPOCH, checkpoint_path, metric_path) #train_loss, val_loss, accuracy
train_loss_list.extend(t_loss)
val_loss_list.extend(v_loss)
val_accuracy_list.extend(v_acc)

Do not exist checkpoint
Start 1


In [ ]:
print(test_loader.dataset.dataframe.columns)

Index(['file_name', 'file_path'], dtype='object')


In [ ]:
results_df = predict(model, test_loader)

In [ ]:
def save_results(results_df):
    current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
    file_name = f"task1_{current_time}.csv"
    results_df.to_csv(file_name, index=False)
    print(f"Results saved to {file_name}")


save_results(results_df)

In [ ]:
!pwd

In [ ]:
MAIN_DIR

'/content/drive/MyDrive/Colab Notebooks/USG_AI'